In [27]:
from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, read_csv, concat
# from pandas import DataFrame, ExcelWriter, date_range, to_datetime
import os
from numpy import isnan
import time

import warnings
warnings.filterwarnings("ignore")

parent = os.path.dirname(os.path.abspath("__file__"))
record_path = os.path.join(parent, "FollowBroker")
if not os.path.isdir(record_path):
    os.makedirs(record_path)

In [19]:
def readDailyRecord(date:datetime):
    ob_f_path = os.path.join(record_path, f'{date.strftime("%Y%m%d")}_OverBuy.csv')
    os_f_path = os.path.join(record_path, f'{date.strftime("%Y%m%d")}_OverSell.csv')
    ob_df, os_df = DataFrame(), DataFrame()
    if os.path.isfile(ob_f_path):
        try:
            ob_df = read_csv(ob_f_path, dtype=str)
            ob_df.columns = ['Ticker','Name', 'Qty', 'Value','Price','Volume', 'Ratio']
        except:
            pass
        try:
            os_df = read_csv(os_f_path, dtype=str)
            os_df.columns = ['Ticker','Name', 'Qty', 'Value','Price','Volume', 'Ratio']
        except:
            pass
    return ob_df, os_df

# First Time Aggregate

In [20]:
ob_dfs = {}
os_dfs = {}
for date in date_range(datetime(2021,1,1), datetime.today()):
    ob_df, os_df = readDailyRecord(date)
    if ob_df.empty and os_df.empty:
        continue
    ob_dfs[date] = ob_df
    os_dfs[date] = os_df

In [21]:
keys = sorted(set(ob_dfs.keys()).intersection(os_dfs.keys()))

In [22]:
ob_dfs[date]

,Ticker,Name,Qty,Value,Price,Volume,Ratio
0,'2618,長榮航,130,"4,171",32.08,124737,0.1
1,'3037,欣興,753,"151,328",200.97,18938,3.98
2,'6026,福邦證,100,"1,698",16.98,1137,8.8
3,'6235,華孚,332,"14,315",43.12,24116,1.38


In [36]:
DataFrame?

In [62]:
Agg_df = DataFrame(index = keys)
Agg_df.index.name = 'Date'

In [69]:
def changeType(x):
    if isinstance(x, float) or isinstance(x, int):
        if isnan(x):
            return 0
        else:
            return x
    elif isinstance(x, str):
        return int(x.replace(',',''))

In [72]:
for i, key in enumerate(keys):
    print(key)
    try:
        tmp_ob = ob_dfs[key].copy(deep=True)
        tmp_os = os_dfs[key].copy(deep=True)

        if not tmp_ob.empty:
            tmp_ob['Date'] = date
            for row in tmp_ob.itertuples():
                if not changeType(row.Qty):continue
                Agg_df.loc[key, str(row.Ticker).replace('\'','') + "_" + row.Name] = changeType(row.Qty)
        if not tmp_os.empty:
            tmp_os['Date'] = date
            for row in tmp_os.itertuples():
                if not changeType(row.Qty):continue
                Agg_df.loc[key, str(row.Ticker).replace('\'','') + "_" + row.Name] = changeType(row.Qty)
    except Exception as e:
        print(e, row)
        break

2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00
2021-02-22 00:00:00
2021-02-23 00:00:00
2021-02-24 00:00:00
2021-02-25 00:00:00
2021-02-26 00:00:00
2021-03-02 00:00:00
2021-03-03 00:00:00
2021-03-04 00:00:00
2021-03-05 00:00:00
2021-03-08 00:00:00
2021-03-09 00:00:00
2021-03-10 00:00:00
2021-03-11 00:00:00
2021-03-12 00:00:00
2021-03-15 00:00:00
2021-03-16 00:00:00
2021-03-17 00:00:00
2021-03-18 00:00:00
2021-03-19 00:00:00
2021-03-22 00:00:00
2021-03-23 00:00:00
2021-03-24 00:00:00


In [73]:
Agg_df#.dropna('all')#[~Agg_df.nan()]

,2603_長榮,2890_永豐金,2317_鴻海,2609_陽明,3006_晶豪科,8021_尖點,2303_聯電,2344_華邦電,3231_緯創,2328_廣宇,...,3520_華盈,4433_興采,5258_虹堡,6163_華電網,5272_笙科,2915_潤泰全,3558_神準,6605_帝寶,00878_國泰永續高股息,4908_前鼎
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,1937.0,2380.0,-271.0,-3225.0,-964.0,-665.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,-7153.0,1100.0,NaN,503.0,NaN,NaN,3076.0,2014.0,308.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-06,480.0,NaN,NaN,1322.0,NaN,NaN,-1977.0,NaN,NaN,1548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-07,3085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-08,-3005.0,233.0,NaN,702.0,NaN,NaN,-250.0,1536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-160.0,NaN,NaN,NaN,NaN,NaN
2022-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-198.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
Agg_df.append(Agg_df.fillna(0).cumsum(), ignore_index=True)

,2603_長榮,2890_永豐金,2317_鴻海,2609_陽明,3006_晶豪科,8021_尖點,2303_聯電,2344_華邦電,3231_緯創,2328_廣宇,...,3520_華盈,4433_興采,5258_虹堡,6163_華電網,5272_笙科,2915_潤泰全,3558_神準,6605_帝寶,00878_國泰永續高股息,4908_前鼎
0,1937.0,2380.0,-271.0,-3225.0,-964.0,-665.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-7153.0,1100.0,NaN,503.0,NaN,NaN,3076.0,2014.0,308.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,480.0,NaN,NaN,1322.0,NaN,NaN,-1977.0,NaN,NaN,1548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3005.0,233.0,NaN,702.0,NaN,NaN,-250.0,1536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-198.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,121.0,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.0,182.0
348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-121.0


In [ ]:
Agg_df.fillna(0).cumsum().apply()